# Ejemplo de uso de los datos

In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn import linear_model


In [2]:
class Limpieza():
    def __init__(self,data):
        self.df=data
        self.df.loc[self.df.vict_age==0,'vict_age']=np.nan
        self.df.loc[(self.df.vict_age<0)|(self.df.vict_age>100),'vict_age']=np.nan
    def porcentaje(self):
        return (self.df).isna().sum()/len(self.df)*100
    def cantidad(self):
        return (self.df).isna().sum()
    def limpieza(self):
        self.df.drop('cross_street',axis=1,inplace=True)
        crm_cd_concat=self.df[['crm_cd_1','crm_cd_2','crm_cd_3','crm_cd_4']].fillna('').astype('string')
        crm_cd_concat=crm_cd_concat['crm_cd_1']+crm_cd_concat['crm_cd_2']+crm_cd_concat['crm_cd_3']+crm_cd_concat['crm_cd_4']
        self.df['crm_cd_concat']=crm_cd_concat
        self.df.drop(['crm_cd_1','crm_cd_2','crm_cd_3','crm_cd_4'],axis=1,inplace=True)
        self.df['weapon_desc'].fillna('NO WEAPON',inplace=True)
        self.df['weapon_used_cd'].fillna('0',inplace=True)
        self.df.loc[self.df.vict_sex.isna(),'vict_sex']='X'
        self.df.loc[self.df.vict_descent.isna(),'vict_descent']='U'
        self.df.loc[self.df.lon==0,['lat','lon']]=np.nan
        #return self.df
        self.df.location = self.df.location.str.strip()
        self.df.location = self.df.location.apply(lambda x: "  ".join(x.split()))
        self.df.location = self.df.location.str.replace(" ", "")
        self.df.location = self.df.location.str.upper()
        directorio = pd.read_excel('Directorio_Lat_Lon.xlsx')
        directorio.location = directorio.location.str.replace(" ", "")
        directorio.location = directorio.location.str.upper()
        loc_lat = dict(zip(directorio.location,directorio.lat))
        loc_lon = dict(zip(directorio.location,directorio.lon))
        self.df.lat.fillna(self.df.location.map(loc_lat),inplace=True)
        self.df.lon.fillna(self.df.location.map(loc_lon),inplace=True)
        self.df.drop('location',axis=1,inplace=True)
        #poner columna de código y luego descripción del código
        self.col_corr('area','area_name')
        self.col_corr('crm_cd','crm_cd_desc')
        self.col_corr('premis_cd','premis_desc')
        self.col_corr('weapon_used_cd','weapon_desc')
        self.col_corr('status','status_desc')
        self.df.loc[self.df.vict_sex=='H','vict_sex']=='X'
        self.df.drop(self.df.loc[self.df.status=='CC'].index,inplace=True)
    def imputar_edad(self):
        ucr=pd.read_csv('ucr.csv',header=None)
        ucr.columns=['Category','Subcategory','Code']
        ucr=ucr.astype({'Code':'string'})
        self.df=self.df.astype({'crm_cd':'string'})
        ucr=dict(zip(ucr['Code'],ucr['Subcategory']))
        self.df['ucr']=self.df.crm_cd.map(ucr)
        self.df.ucr=self.df.ucr.fillna('Other')
        imputar=self.df.drop(['dr_no', 'date_rptd', 'date_occ','area',
                         'rpt_dist_no','part_1_2','crm_cd','mocodes','premis_cd',
                         'weapon_used_cd','crm_cd_concat','rpt_dist_no'],axis=1).copy()
        dum1=pd.get_dummies(imputar['status']).drop('JO',axis=1)
        imputar.drop('status',axis=1,inplace=True)
        dum2=pd.get_dummies(imputar['vict_sex']).drop('X',axis=1)
        imputar.drop('vict_sex',axis=1,inplace=True)
        dum3=pd.get_dummies(imputar['vict_descent']).drop('Z',axis=1)
        imputar.drop('vict_descent',axis=1,inplace=True)
        dum4=pd.get_dummies(imputar['ucr']).drop('Other',axis=1)
        imputar.drop('ucr',axis=1,inplace=True)
        imputar=pd.concat([imputar,dum1,dum2,dum3,dum4],axis=1)
        scaler = MinMaxScaler()
        imputar=pd.DataFrame(scaler.fit_transform(imputar),columns=imputar.columns)
        X=imputar[~imputar.vict_age.isna()]
        y=X.vict_age.values
        X=X.drop('vict_age',axis=1).values
        neigh = KNeighborsRegressor(n_neighbors=5)
        neigh.fit(X, y.reshape(-1, 1))
        estimados=neigh.predict(imputar.loc[imputar.vict_age.isna(),imputar.columns!='vict_age'].values)
        estimados=[x[0] for x in estimados]
        estimados=pd.Series(index=imputar.loc[imputar.vict_age.isna(),'vict_age'].index,data=estimados)
        imputar.loc[imputar.vict_age.isna(),'vict_age']=estimados
        self.df['vict_age']=pd.DataFrame(scaler.inverse_transform(imputar),columns=imputar.columns)['vict_age']
    
    
    def imputar_edad_2(self):
        ucr=pd.read_csv('ucr.csv')
        ucr=ucr.astype({'Code':'string'})
        self.df=self.df.astype({'crm_cd':'string'})
        ucr=dict(zip(ucr['Code'],ucr['Subcategory']))
        self.df['ucr']=self.df.crm_cd.map(ucr)
        self.df.ucr=self.df.ucr.fillna('Other')
        imputar=self.df.drop(['date_rptd', 'date_occ','area',
                         'rpt_dist_no','part_1_2','crm_cd','mocodes','premis_cd',
                         'weapon_used_cd','crm_cd_concat','rpt_dist_no'],axis=1).copy()#'dr_no', 
        dum1=pd.get_dummies(imputar['status']).drop('JO',axis=1)
        imputar.drop('status',axis=1,inplace=True)
        dum2=pd.get_dummies(imputar['vict_sex']).drop('X',axis=1)
        imputar.drop('vict_sex',axis=1,inplace=True)
        dum3=pd.get_dummies(imputar['vict_descent']).drop('Z',axis=1)
        imputar.drop('vict_descent',axis=1,inplace=True)
        dum4=pd.get_dummies(imputar['ucr']).drop('Other',axis=1)
        imputar.drop('ucr',axis=1,inplace=True)
        imputar=pd.concat([imputar,dum1,dum2,dum3,dum4],axis=1)
        
        
        X=imputar[~imputar.vict_age.isna()]
        y=X.vict_age.values
        X=X.drop('vict_age',axis=1).values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        sc_X = MinMaxScaler()
        sc_y = MinMaxScaler()
        X_train=sc_X.fit_transform(X_train)
        y_train=sc_y.fit_transform(y_train.reshape(-1, 1))
        y_test=sc_y.transform(y_test.reshape(-1,1))
        X_test=sc_X.transform(X_test)
        
        clf = Ridge(alpha=1.0)
        clf.fit(X_train, y_train)
        y_train_p=clf.predict(X_train)
        #scores = cross_val_score(clf, X_train, y_train, cv=5)
        #y_train_p=scores.score(X_train)
        #print(r2_score(y_train, y_train_p))
        
        y_test_p=clf.predict(X_test)
        print(r2_score(y_test, y_test_p))
        
        print(mean_squared_error(y_train, y_train_p))
        print(mean_squared_error(y_test, y_test_p))
        
        
        
    def imputar_premis(self):
        self.df=self.df.astype({'premis_cd':'string'})
        imputar=self.df.drop([ 'date_rptd', 'date_occ','area',
                         'rpt_dist_no','part_1_2','crm_cd','mocodes',
                         'weapon_used_cd','crm_cd_concat','rpt_dist_no'],axis=1).copy()#'dr_no',
        dum1=pd.get_dummies(imputar['status']).drop('JO',axis=1)
        imputar.drop('status',axis=1,inplace=True)
        dum2=pd.get_dummies(imputar['vict_sex']).drop('X',axis=1)
        imputar.drop('vict_sex',axis=1,inplace=True)
        dum3=pd.get_dummies(imputar['vict_descent']).drop('Z',axis=1)
        imputar.drop('vict_descent',axis=1,inplace=True)
        dum4=pd.get_dummies(imputar['ucr']).drop('Other',axis=1)
        imputar.drop('ucr',axis=1,inplace=True)
        imputar=pd.concat([imputar,dum1,dum2,dum3,dum4],axis=1)
        
        sc_x = MinMaxScaler()
        X=imputar.loc[~self.df.premis_cd.isna(),imputar.columns!='premis_cd']
        y=imputar.loc[~self.df.premis_cd.isna(),'premis_cd']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        X_train=pd.DataFrame(sc_x.fit_transform(X_train),columns=X.columns)
        X_test=pd.DataFrame(sc_x.transform(X_test),columns=X.columns)
        
        
        
        
        neigh = KNeighborsClassifier(n_neighbors=4)
        neigh.fit(X_train,y_train)
        y_predict=neigh.predict(X_test)
        return accuracy_score(y_test, y_predict)
        
        
        #estimados=neigh.predict(imputar.loc[imputar.premis_cd.isna(),imputar.columns!='premis_cd'].values)
        
        #estimados=pd.Series(index=imputar.loc[imputar.premis_cd.isna(),'premis_cd'].index,data=estimados)
        #self.df.loc[imputar.premis_cd.isna(),'premis_cd']=estimados
        #self.df['premis_cd']=pd.DataFrame(scaler.inverse_transform(imputar.drop('premis_cd')),columns=imputar.columns)['premis_cd']
    
    def imputar_mocodes(self):
        imputar=self.df.drop([ 'date_rptd', 'date_occ','area',
                         'rpt_dist_no','part_1_2','crm_cd','premis_cd',
                         'weapon_used_cd','crm_cd_concat','rpt_dist_no'],axis=1).copy()#'dr_no',
        dum1=pd.get_dummies(imputar['status']).drop('JO',axis=1)
        imputar.drop('status',axis=1,inplace=True)
        dum2=pd.get_dummies(imputar['vict_sex']).drop('X',axis=1)
        imputar.drop('vict_sex',axis=1,inplace=True)
        dum3=pd.get_dummies(imputar['vict_descent']).drop('Z',axis=1)
        imputar.drop('vict_descent',axis=1,inplace=True)
        dum4=pd.get_dummies(imputar['ucr']).drop('Other',axis=1)
        imputar.drop('ucr',axis=1,inplace=True)
        imputar=pd.concat([imputar,dum1,dum2,dum3,dum4],axis=1)
        
        scaler = MinMaxScaler()
        
        
        
        sc_x = MinMaxScaler()
        X=imputar.loc[~imputar.mocodes.isna(),imputar.columns!='mocodes']
        y=imputar.loc[~imputar.mocodes.isna(),'mocodes']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        X_train=pd.DataFrame(sc_x.fit_transform(X_train),columns=X.columns)
        
        
        
        
        
        neigh = KNeighborsClassifier(n_neighbors=3, scoring='r2')
        neigh.fit(X_train,y_train)
        y_predict=neigh.predict(X_test)
        print(accuracy_score(y_test, y_predict))
        
        
       
    def col_corr(self,codigo,descripcion):
        dicc=dict(zip(self.df[codigo],self.df[descripcion]))
        w = csv.writer(open(codigo+".csv", "w"))
        for key, val in dicc.items():
            w.writerow([key, val])
        self.df.drop(descripcion,axis=1,inplace=True)
        
    def actualizar(self):
        return self.df
    def categoricas(self):
        features=['rpt_dist_no','premis_cd','weapon_used_cd']
        for feature in features:
            aux = self.df[feature].value_counts(True)
            ls_categories = [category for category, freq in aux.items() if freq > 0.03]
            self.df[feature] = self.df[feature].map(lambda x: x if x in ls_categories else "Others")

In [3]:
#la api solo trae de 999 en 999
#histograma de edades
'''from Class_GetData import Get_Data
Data= Get_Data()
Data.create_df()
Data.infer_dtypes()
df= Data.df.copy()
df.head()'''

'from Class_GetData import Get_Data\nData= Get_Data()\nData.create_df()\nData.infer_dtypes()\ndf= Data.df.copy()\ndf.head()'

In [4]:
#df.to_csv('datos_prueba.csv',index=False)

In [3]:
df2=pd.read_csv('datos_prueba.csv',index_col=0)

In [49]:
#from Class_ContGraphs import Graphs_Cont

In [50]:
#graphs= Graphs_Cont()

In [51]:
#graphs.box(df)

In [150]:
#graphs.heatMap(df)

In [147]:
#temporal=df[(df.lat<34.5)&(df.lon>-119)&(df.lon<-118)]

In [151]:
#graphs.scatter(temporal)

In [6]:
#graphs.hist(df)

In [52]:
#df.isna().sum()

In [35]:
#from disc_graph import Discrete

In [36]:
#gen = Discrete(df)
#gen.pie_chart()

In [37]:
#gen.freq_table()

In [38]:
#gen.bar_chart()

In [39]:
#gen.time_series()


## Empieza mi parte

In [4]:
nulos=Limpieza(df2)

In [21]:
nulos=Limpieza(df)


In [11]:
print(nulos.porcentaje())

date_rptd          0.000000
date_occ           0.000000
time_occ           0.000000
area               0.000000
area_name          0.000000
rpt_dist_no        0.000000
part_1_2           0.000000
crm_cd             0.000000
crm_cd_desc        0.000000
mocodes           13.746887
vict_age          24.200176
vict_sex          13.108008
vict_descent      13.108751
premis_cd          0.001188
premis_desc        0.050938
weapon_used_cd    65.185404
weapon_desc       65.185404
status             0.000000
status_desc        0.000000
crm_cd_1           0.001188
location           0.000000
lat                0.000000
lon                0.000000
crm_cd_2          92.516562
cross_street      83.664629
crm_cd_3          99.747240
crm_cd_4          99.992426
dtype: float64


In [8]:
print(nulos.cantidad())

date_rptd             0
date_occ              0
time_occ              0
area                  0
rpt_dist_no           0
part_1_2              0
crm_cd                0
mocodes           92567
vict_age              0
vict_sex              0
vict_descent          0
premis_cd             8
weapon_used_cd        0
status                0
lat                   0
lon                   0
crm_cd_concat         0
ucr                   0
dtype: int64


In [5]:
nulos.limpieza()

In [6]:
nulos.categoricas()

In [14]:
nulos.porcentaje()

date_rptd          0.000000
date_occ           0.000000
time_occ           0.000000
area               0.000000
rpt_dist_no        0.000000
part_1_2           0.000000
crm_cd             0.000000
mocodes           13.746928
vict_age          24.200248
vict_sex           0.000000
vict_descent       0.000000
premis_cd          0.000000
weapon_used_cd     0.000000
status             0.000000
lat                0.000000
lon                0.000000
crm_cd_concat      0.000000
dtype: float64

In [18]:
nulos.imputar_edad_2()

0.06923787643537138
0.023826411491622602
0.023935138905522655


In [12]:
nulos.imputar_edad_2()

0.06770089814716163
0.06923787643537138
0.023826411491622602
0.023935138905522655


In [15]:
nulos.imputar_premis()

0.22405548295117025


/home/mariano/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [12]:
nulos.imputar_mocodes()

0.00522842056932966


In [32]:
df.columns

Index(['date_rptd', 'date_occ', 'time_occ', 'area', 'rpt_dist_no', 'part_1_2',
       'crm_cd', 'mocodes', 'vict_age', 'vict_sex', 'vict_descent',
       'premis_cd', 'weapon_used_cd', 'status', 'lat', 'lon', 'crm_cd_concat'],
      dtype='object')

In [9]:
#nulos.imputar_edad()
#df.to_csv('datos_limpios.csv',index=False)

In [16]:
df=pd.read_csv('datos_limpios.csv')

In [17]:
nulos.porcentaje()

date_rptd          0.000000
date_occ           0.000000
time_occ           0.000000
area               0.000000
rpt_dist_no        0.000000
part_1_2           0.000000
crm_cd             0.000000
mocodes           13.746887
vict_age           0.000000
vict_sex           0.000000
vict_descent       0.000000
premis_cd          0.000000
weapon_used_cd     0.000000
status             0.000000
lat                0.000000
lon                0.000000
crm_cd_concat      0.000000
ucr                0.000000
dtype: float64

In [18]:
df.set_index('dr_no',inplace=True)
df['premis_cd']=df2['premis_cd']

In [23]:
nulos.imputar_premis()

0.3195962536137183

In [9]:
df2.columns

Index(['date_rptd', 'date_occ', 'time_occ', 'area', 'rpt_dist_no', 'part_1_2',
       'crm_cd', 'mocodes', 'vict_age', 'vict_sex', 'vict_descent',
       'premis_cd', 'weapon_used_cd', 'status', 'lat', 'lon', 'crm_cd_concat'],
      dtype='object')

In [8]:
columnas=['rpt_dist_no','premis_cd','weapon_used_cd']
for column in columnas:
    print(len(df2[column].unique()))
    print(df2[column].value_counts())

1
Others    673365
Name: rpt_dist_no, dtype: int64
8
Others    177451
101.0     169935
501.0     114177
502.0      82346
108.0      48443
203.0      31022
102.0      29471
122.0      20520
Name: premis_cd, dtype: int64
4
0         438936
400.0     125855
Others     84451
500.0      24123
Name: weapon_used_cd, dtype: int64


In [116]:
vivienda=[119,120,121,145,146.0,150.0,501.0, 502.0, 504.0,507.0, 508.0, 509.0, 510.0,511,515.0, 516.0,707]
lote_aband=[107.0,118,506.0,518.0]
calle=[101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 109.0, 110.0,116,117,
      124.0, 125.0,128,136.0, 137.0,152,154,158,415,705.0,748.0]
gobierno=[725.0, 726.0,240.0,732.0,752,753,756.0]
estacionamiento=[123,142,156,204.0, 213.0]
gasolineria=[301]
establecimiento=[151.0,157.0,238,241,254,302,303,304.0, 305.0,702]
establecimiento_publico=[201.0, 202.0, 203.0,205.0, 206.0,208,209.0, 210.0, 211.0,217.0, 218.0,
                        219.0, 220.0, 221.0, 222.0, 223.0, 224.0, 225.0,228,233.0,242,244.0, 245.0,
                        247.0, 248.0, 249.0,250.0, 251.0, 252.0,255,401.0, 402.0, 404.0, 405.0,
                         406.0, 407.0, 408.0,409.0, 410.0, 411.0, 412.0, 413.0, 414.0,417.0, 418.0,703,726.0,
                         874.0, 875.0, 876.0, 877.0, 878.0, 879.0, 880.0, 882.0, 883.0, 884.0,
                         885.0, 889.0, 890.0, 891.0, 892.0, 893.0, 894.0, 895.0, 896.0, 897.0,
                         898.0, 900.0, 901.0, 902.0, 903.0, 904.0, 905.0, 906.0, 907.0, 908.0,
                         909.0, 910.0, 911.0, 912.0, 913.0, 915.0, 916.0, 917.0, 918.0, 919.0,
                         920.0, 921.0, 922.0, 931.0, 932.0, 933.0, 934.0, 935.0, 936.0, 937.0,
                         940.0, 941.0, 942.0, 943.0, 944.0, 945.0, 946.0, 947.0, 948.0, 949.0,
                         950.0, 951.0, 952.0, 953.0, 954.0, 956.0, 957.0, 958.0, 961.0, 962.0,
                         963.0, 964.0, 965.0, 966.0, 967.0, 968.0, 969.0, 970.0, 971.0,
                        ]
estacion=[129,135]
nocturno=[207,706.0,733.0,735.0]
hotel=[503,505,519.0]
transport=[111.0, 112.0, 113.0, 114.0, 115.0,122,126,212.0, 214.0, 215.0,801.0, 802.0,804,809.0, 810.0,
          811.0, 834.0, 835.0, 836.0, 868.0, 869.0, 870.0, 871.0, 872.0, 873.0]
basurero=[127]
escaleras=[138,143,727]
recreacion=[139,108,147.0,712.0,713,144,714.0, 715.0, 716.0, 717.0, 718.0,711,734.0, 736.0, 737.0, 738.0, 739.0,
           742.0, 743.0,754,757.0, 758.0]
balcon=[140]
cementerio=[141]
public_bathroom=[148.0,709]
naturaleza=[149,155,745.0]
mercado=[216]
medico=[227.0,234.0, 235.0, 236.0, 237.0,239,246.0,253,403,701.0,719,755]
financiero=[229,601,602.0, 603.0, 604.0, 605.0, 606.0, 607.0, 608.0]
asilo=[230.0, 512.0,514,517]
escuela=[231.0,704.0,720.0, 721.0, 722.0, 723.0,724.0,729]
otro=[232.0,245.0,513,710,744.0,746.0]
iglesia=[709.0, 730.0, 731.0,740]
internet=[750.0, 751.0]

In [13]:
lugares=pd.read_csv('premis_cd.csv',header=None)
lugares.columns=['cod','lugar']

In [138]:
n=310
m=n+10

In [139]:
list(lugares.sort_values('cod')[n:]['cod'])

[nan, nan, nan, nan, nan, nan, nan]

In [140]:
lugares.sort_values('cod')[n:m]

,cod,lugar
287,NaN,NaN
308,NaN,NaN
309,NaN,NaN
310,NaN,NaN
312,NaN,NaN
313,NaN,NaN
315,NaN,NaN


In [56]:
len(lugares.sort_values('cod'))

317

In [90]:
lugares[lugares.cod==711]

,cod,lugar
211,711.0,"ARCADE,GAME ROOM/VIDEO GAMES (EXAMPLE CHUCKIE ..."
